In [6]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [12]:
# MNIST 데이터셋 로드
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다. 
# TODO 채널은 무슨 용도?
print(x_train.shape, x_test.shape)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [13]:
# tf.data를 사용하여 데이터셋을 섞고 배치 생성
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [21]:
# 케라스(Keras)의 모델 SubClassing API를 사용하여 tf.keras 모델 생성
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # CNN
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [22]:
# Optimizer & Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [23]:
# 모델의 손실과 성능을 측정할 지표를 선택
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [24]:
@tf.function
def train_step(images, labels):
    # tf.GradientTape를 사용하여 모델을 훈련
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [25]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [26]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.13839580118656158, 정확도: 95.85166931152344, 테스트 손실: 0.06694156676530838, 테스트 정확도: 97.72999572753906
에포크: 2, 손실: 0.09115339815616608, 정확도: 97.23416900634766, 테스트 손실: 0.061488404870033264, 테스트 정확도: 97.98999786376953
에포크: 3, 손실: 0.06823190301656723, 정확도: 97.92444610595703, 테스트 손실: 0.05926074460148811, 테스트 정확도: 98.13333129882812
에포크: 4, 손실: 0.054067209362983704, 정확도: 98.3499984741211, 테스트 손실: 0.061591558158397675, 테스트 정확도: 98.16999816894531
에포크: 5, 손실: 0.04529454931616783, 정확도: 98.61033630371094, 테스트 손실: 0.0628715232014656, 테스트 정확도: 98.1760025024414
